In [14]:
import numpy as np
import torchvision
import torchvision.datasets as datasets
from sklearn.decomposition import PCA
import scipy.special as sp
import random

In [9]:
from sklearn.mixture import GaussianMixture

### Part I: Train GMM with no dataset argumentation ###

In [10]:
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
mnist_trainset =  datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
raw_trainX = mnist_trainset.data.numpy()
raw_trainY = mnist_trainset.targets.numpy()
raw_testX = mnist_testset.data.numpy()
raw_testY = mnist_testset.targets.numpy()  

/home/kent/.local/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


#### Model 1: We assume each class of digits in mnist shares some features follows Gaussian distribution, hence we feed in these classes and use trained GMMs for K classes to predict label of unseen images.

In [11]:
# use pca to reduce the dimesion of the dataset, original data size is 28 * 28
pca = PCA(n_components=60)
trainX = raw_trainX.reshape(raw_trainX.shape[0], -1)
trainX = pca.fit_transform(trainX)
testX = raw_testX.reshape(raw_testX.shape[0], -1)
testX = pca.transform(testX)

In [12]:
print(trainX.shape)

(60000, 60)


In [25]:
K=8             # K decides how many classes is feed in the model in stage 1 

stage1_classes = random.sample(range(10), K)
model = []

print(stage1_classes)
# stage 1 training
for i in stage1_classes:
    trainX_i = trainX[raw_trainY == i]
    model.append(GaussianMixture(n_components=5).fit(trainX_i[:3000]))

[6, 4, 8, 2, 7, 9, 0, 5]


In [30]:
print(len(model))
print(model[1])

8
GaussianMixture(n_components=5)


In [36]:
def GMMs_predict(x, gmms, k, treshold):
    predict = np.zeros((x.shape[0], k), dtype=float)

    # use pretrained model to predict x
    for i in range(k):
        predict[i] = gmms[i].score(x)
    
    predict = sp.softmax(predict, axis=1)
    max_class = np.amax(predict, axis=1)
    return np.where(max_class > treshold, np.argmax(predict), -1)

In [37]:
# stage 2 unsupervised training:
predict = GMMs_predict(trainX, model, K, 0.5)

# use data that cannot be classified to train the rest classes
unclassified = trainX[predict == -1]

        
    


    